# 结合代理和矢量存储

本手册介绍了如何组合代理和矢量存储。这样做的用例是，您已经将数据摄取到矢量存储中，并希望以代理的方式与之交互。

推荐的方法是创建`RetrievalQA`，然后将其用作整个代理中的工具。让我们看看下面是怎么做的。您可以对多个不同的矢量db执行此操作，并使用代理作为在它们之间进行路由的方法。有两种不同的方法可以做到这一点——你可以让代理使用向量存储作为普通工具，或者你可以设置`return_direct=True`来真正地将代理用作路由器。

## Create the vector store

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.chat_models import ChatOllama

# llm = OpenAI(temperature=0)
llm = ChatOllama(model="qwen:32b-chat-v1.5-q5_K_M")

In [2]:
from pathlib import Path

doc_path = str(Path(".").absolute() / "流浪地球.txt")
print(doc_path)

/home/huangwei1/repo/langchain/cookbook/流浪地球.txt


In [16]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings


loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OllamaEmbeddings(model="znbang/bge:large-zh-v1.5-f16")
docsearch = Chroma.from_documents(texts, embeddings, collection_name="LiuCixin-WanderingEarth")

In [8]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [9]:
from langchain_community.document_loaders import WebBaseLoader

In [18]:
loader = WebBaseLoader("http://71.136.100.40:3000/?wework_cfm_code=MoL9rcXqCEX2sOXsQF8QRLWWz2NKlLNTXHflPKSOu22ZGPZv%2B4%2BI86KrzytAQ%2F%2FePVNxWUutHYlKRjEP20YhSI0%3D#/intro/product")

In [19]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="Songyin-OrcaStudio")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [12]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [13]:
tools = [
    Tool(
        name="流浪地球 问答系统",
        func=state_of_union.run,
        description="当您需要回答有关最新流浪地球的问题时，非常有用。输入应该是一个完全形成的问题。",
    ),
    Tool(
        name="松应科技 OrcaStudio 问答系统",
        func=ruff.run,
        description="当您需要回答有关松应科技 OrcaStudio（一种3D内容协作平台）的问题时非常有用。输入应该是一个完全形成的问题。",
    ),
]

In [14]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

/home/huangwei1/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [17]:
agent.run(
    "流浪地球是发生在哪一年的故事？"
)



> Entering new AgentExecutor chain...
 我需要使用流浪地球问答系统来查找这个信息。
Action: 流浪地球 问答系统
Action Input: 请问流浪地球故事发生的时间背景是什么时候
Observation: 《流浪地球》的故事发生在不远的未来，具体时间点未在原著小说中明确提及。随着太阳老化并膨胀成红巨星，人类面临生存危机，启动了“流浪地球计划”，这个大规模的人类工程持续了几千年，目标是将地球推离太阳系，寻找新的恒星家园。故事中的关键事件如木星引力助推、地球发动机的运行等，发生在计划实施过程中的某个阶段。
Thought: 尽管没有一个确切的年份，但我们可以了解到《流浪地球》的故事发生在未来的一个未指明的时间点。
Final Answer: 流浪地球的故事发生在未指定的未来时间点，具体年代未在原著或电影中明确提及。

> Finished chain.


'流浪地球的故事发生在未指定的未来时间点，具体年代未在原著或电影中明确提及。'

In [21]:
agent.run("松应科技3D内容协作平台如何降低协作成本？")



> Entering new AgentExecutor chain...
 需要使用松应科技OrcaStudio问答系统来回答关于该产品降低成本的具体方式。
Action: 松应科技 OrcaStudio 问答系统
Action Input: 松应科技的3D内容协作平台是如何帮助降低团队协作成本的

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2



Observation: 松应科技的3D内容协作平台通过提供一个集成化的环境，使得团队成员可以共同创作、编辑和审阅3D内容，无需传统的面对面会议或物理模型制作。这样可以减少沟通成本，提升工作流程效率，并且减少了因信息传递不准确或版本控制不当导致的错误和返工。

具体来说，这个平台可能有以下功能来帮助降低团队协作成本：

1. **实时协作**：团队成员可以同时在线编辑同一份3D项目，这降低了需要等待他人完成工作的延迟，提高了生产力。

2. **云存储与版本控制**：所有文件都储存在云端，并且自动保存每个版本。这样可以避免数据丢失，减少因错误或不满意的结果而重新开始的工作量。

3. **远程工作支持**：团队成员无需在同一地点，只要能上网就能参与项目，这节省了旅行时间和成本，也适应了现代的分布式工作模式。

4. **可视化交流**：通过平台内置的实时评论和标记功能，团队可以更直观地讨论设计细节，降低了语言沟通可能产生的误解。

5. **资源库与重用**：平台可能会提供一个共享的资源库，团队成员可以从中获取、重用或修改已有的3D模型和素材，减少了从头创建的时间成本。

6. **自动化工作流程**：通过集成各种工具和插件，可能能自动完成一些繁琐的任务，如渲染预览、模型优化等，进一步提升了效率。

然而，具体到松应科技的平台如何实现这些功能，以及实际效果，可能需要查看该公司的详细介绍或亲自试用其产品来获取更准确的信息。
Thought: 我已经得到了关于松应科技3D内容协作平台降低协作成本的主要方式。
Final Answer: 松应科技3D内容协作平台通过提供实时协作、云存储与版本控制、远程工作支持、可视化交流、资源库与重用及自动化工作流程等功能来降低团队协作成本。这减少了沟通和传输错误，提升了工作效率，并适应了分布式工作模式，降低了物理会议和模型制作的成本。具体功能的实现细节可能需要参考官方介绍或实际试用产品。

> Finished chain.


'松应科技3D内容协作平台通过提供实时协作、云存储与版本控制、远程工作支持、可视化交流、资源库与重用及自动化工作流程等功能来降低团队协作成本。这减少了沟通和传输错误，提升了工作效率，并适应了分布式工作模式，降低了物理会议和模型制作的成本。具体功能的实现细节可能需要参考官方介绍或实际试用产品。'

## Use the Agent solely as a router

如果您打算使用代理作为路由器，并且只是想直接返回RetrievalQAChain的结果，您也可以设置`return_direct=True`。

注意，在上面的示例中，代理在查询RetrievalQAChain之后做了一些额外的工作。你可以避免这个，直接返回结果。

In [22]:
tools = [
    Tool(
        name="流浪地球 问答系统",
        func=state_of_union.run,
        description="当您需要回答有关最新流浪地球的问题时，非常有用。输入应该是一个完全形成的问题。",
        return_direct=True,
    ),
    Tool(
        name="松应科技 OrcaStudio 问答系统",
        func=ruff.run,
        description="当您需要回答有关松应科技 OrcaStudio（一种3D内容协作平台）的问题时非常有用。输入应该是一个完全形成的问题。",
        return_direct=True,
    ),
]

In [23]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
agent.run(
    "流浪地球是发生在哪一年的故事？"
)



> Entering new AgentExecutor chain...
 我需要使用流浪地球问答系统来查找这个故事发生的时间线。
Action: 流浪地球 问答系统
Action Input: 请问《流浪地球》的故事发生在哪一年
Observation: 在刘慈欣所著的科幻小说《流浪地球》中，并没有明确提到故事发生的具体年份。这本小说更多地聚焦于地球在未来面临的灾难以及人类为了生存而采取的极端措施，而非具体的年代背景。如果你对时间设定有特定的需求，可能需要根据小说中的情境和科技发展程度进行推测或参考其他相关资料。


> Finished chain.


'在刘慈欣所著的科幻小说《流浪地球》中，并没有明确提到故事发生的具体年份。这本小说更多地聚焦于地球在未来面临的灾难以及人类为了生存而采取的极端措施，而非具体的年代背景。如果你对时间设定有特定的需求，可能需要根据小说中的情境和科技发展程度进行推测或参考其他相关资料。'

In [26]:
agent.run("松应科技3D内容协作平台如何降低协作成本？")



> Entering new AgentExecutor chain...
 我需要调用OrcaStudio的问答系统来获取关于其如何降低成本的信息。
Action: 松应科技 OrcaStudio 问答系统
Action Input: 如何通过OrcaStudio降低3D内容协作的成本

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2



Observation: 使用OrcaStudio来降低3D内容协作成本的主要方法是通过提供一个集成的工作流程，让团队成员可以同时在一个共享的虚拟空间中工作，这样可以减少沟通成本和版本控制问题。以下是一些具体的步骤或特性可以帮助你实现这一点：

1. **实时协作**：OrcaStudio允许多人同时编辑同一个3D项目，减少了来回发送文件和更新版本的时间。

2. **云服务支持**：通过云端存储，团队成员可以从任何地方访问最新版本的项目，无需本地高配置硬件，节省设备成本。

3. **高效的沟通工具**：集成的消息系统和评论功能使得团队成员可以即时反馈和解决问题，提高工作效率。

4. **资产管理和版本控制**：OrcaStudio提供了强大的资产管理器，可以帮助你追踪每个版本的变化，便于回滚到任何历史状态，避免因误操作或丢失工作而导致的时间浪费。

5. **自动化工作流程集成**：可以与其他3D工具无缝连接，通过脚本和插件自动化重复任务，降低人为错误和时间消耗。

6. **培训和支持**：OrcaStudio提供的教程、文档和社区支持可以帮助团队快速上手，减少学习成本。

7. **视觉化协作**：项目成员可以看到彼此在模型上的工作区域，这有助于理解整体进度和他人的工作内容，从而更有效地协调。

为了具体操作，请访问OrcaStudio的官方网站或查看其用户手册以获取详细的步骤指导。如果你对某个特定功能有疑问，可以继续提问，我会尽量提供帮助。


> Finished chain.


'使用OrcaStudio来降低3D内容协作成本的主要方法是通过提供一个集成的工作流程，让团队成员可以同时在一个共享的虚拟空间中工作，这样可以减少沟通成本和版本控制问题。以下是一些具体的步骤或特性可以帮助你实现这一点：\n\n1. **实时协作**：OrcaStudio允许多人同时编辑同一个3D项目，减少了来回发送文件和更新版本的时间。\n\n2. **云服务支持**：通过云端存储，团队成员可以从任何地方访问最新版本的项目，无需本地高配置硬件，节省设备成本。\n\n3. **高效的沟通工具**：集成的消息系统和评论功能使得团队成员可以即时反馈和解决问题，提高工作效率。\n\n4. **资产管理和版本控制**：OrcaStudio提供了强大的资产管理器，可以帮助你追踪每个版本的变化，便于回滚到任何历史状态，避免因误操作或丢失工作而导致的时间浪费。\n\n5. **自动化工作流程集成**：可以与其他3D工具无缝连接，通过脚本和插件自动化重复任务，降低人为错误和时间消耗。\n\n6. **培训和支持**：OrcaStudio提供的教程、文档和社区支持可以帮助团队快速上手，减少学习成本。\n\n7. **视觉化协作**：项目成员可以看到彼此在模型上的工作区域，这有助于理解整体进度和他人的工作内容，从而更有效地协调。\n\n为了具体操作，请访问OrcaStudio的官方网站或查看其用户手册以获取详细的步骤指导。如果你对某个特定功能有疑问，可以继续提问，我会尽量提供帮助。'

## 多跳向量存储推理

由于向量存储很容易作为代理中的工具使用，因此使用现有代理框架很容易使用依赖于向量存储的多跳问题的答案。

In [27]:
tools = [
    Tool(
        name="流浪地球 问答系统",
        func=state_of_union.run,
        description="当您需要回答有关最新流浪地球的问题时，非常有用。输入应该是一个完整的问题，不要引用之前对话中任何模糊的代词。",
    ),
    Tool(
        name="松应科技 OrcaStudio 问答系统",
        func=ruff.run,
        description="当您需要回答有关松应科技 OrcaStudio（一种3D内容协作平台）的问题时非常有用。输入应该是一个完整的问题，不要引用之前对话中任何模糊的代词。",
    ),
]

In [28]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [31]:
agent.run(
    "OrcaStudio的产品理念是什么? 在流浪地球中，刘启是否使用过OrcaStudio？"
)



> Entering new AgentExecutor chain...
 针对第一个问题，我需要利用松应科技OrcaStudio问答系统来获取产品理念的相关信息。而对于第二个问题，由于它涉及到电影《流浪地球》中的情节，这超出了松应科技产品的范畴，因此我将不使用工具直接根据普遍知识作答。
Action: 松应科技 OrcaStudio 问答系统
Action Input: OrcaStudio的产品理念是什么

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2



Observation: 我不知道OrcaStudio的产品理念是什么，可以提供更多信息吗？
Thought: 对于第一个问题，我没有得到有用的信息，而对于第二个问题，我可以直接作答。
Final Answer: OrcaStudio的产品理念可能是创新、高效和协作，但具体信息可能需要访问其官方网站或相关营销材料以获取准确答案。在电影《流浪地球》中，刘启并没有使用OrcaStudio，因为该影片的故事背景设定在未来，而OrcaStudio是现实中的一个3D内容协作平台，两者不在同一时空背景下，且电影中的科技设备通常是虚构的。

> Finished chain.


'OrcaStudio的产品理念可能是创新、高效和协作，但具体信息可能需要访问其官方网站或相关营销材料以获取准确答案。在电影《流浪地球》中，刘启并没有使用OrcaStudio，因为该影片的故事背景设定在未来，而OrcaStudio是现实中的一个3D内容协作平台，两者不在同一时空背景下，且电影中的科技设备通常是虚构的。'